# Compare image gradients with or w/o gaussian filtering (high pass)




## Load libraries

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os,sys
import re

sys.dont_write_bytecode = True

In [ ]:
import pandas as pd
# import geopandas as gpd
import joblib

import numpy as np
import matplotlib.pyplot as plt
import cv2

from pprint import pprint
from pathlib import Path

import pdb

In [ ]:
import holoviews as hv
import xarray as xr

from holoviews import opts
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.streams import Stream, param
from holoviews import streams
import geoviews as gv
import geoviews.feature as gf
from geoviews import tile_sources as gvts


# import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf

hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'

# Dashboards
import param as pm, panel as pn
pn.extension()

In [ ]:
# Geoviews visualization default options
H,W, = 250,250
opts.defaults(
    opts.RGB(height=H, width=W, tools=['hover'], active_tools=['wheel_zoom']),
    opts.Image(height=H, width=W, tools=['hover'], active_tools=['wheel_zoom'], framewise=True),#axiswise=True ),
    opts.Points( tools=['hover'], active_tools=['wheel_zoom']),
    opts.Curve( tools=['hover'], active_tools=['wheel_zoom'], padding=0.1),

)

In [ ]:
this_nb_path = Path(os.getcwd())
ROOT = this_nb_path.parent
SCRIPTS = ROOT/'codes'
paths2add = [this_nb_path, SCRIPTS]

print("Project root: ", str(ROOT))
print("this nb path: ", str(this_nb_path))
print('Scripts folder: ', str(SCRIPTS))

for p in paths2add:
    if str(p) not in sys.path:
        sys.path.insert(0, str(p))
        print(str(p), "added to the path\n")
        
# print(sys.path)

In [ ]:
import ipywidgets
from ipywidgets import interact
def f(x):
    return x

interact(f, x=10)

In [ ]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

## Tile n-dim array to (n+1) dim
Use `np.broadcast_to(arr, (n_times, *arr.shape))` for tiling arr n_times in the first new-axis dimension

- Ref.
    - https://stackoverflow.com/a/54269982

In [ ]:
def prepend_dim(t, n_times=1, in_place=False):
    """
    Prepend dimension (of size=1) to the input tensor `t`'s dimension
    n_times
    """
    if in_place:
        _t = torch.empty_like(t)
        _t.copy_(t)
    else: 
        _t = t
        
    for i in range(n_times):
        _t.unsqueeze_(dim=0)
    return _t

In [ ]:
x = transforms.ToTensor()(train_x)
x.unsqueeze_(dim=0)
x.shape


In [ ]:
conv = nn.Conv2d(in_channels=x.shape[1], 
                 out_channels=2,
             xw    kernel_size=3,
                 padding=1,)

             
     

In [ ]:
## Testing with x,y gradient filters 
grad_x = [[-1,0,1], [-1,0,1], [-1,0,1]]
grad_x = np.asarray(grad_x, dtype=np.float32)
grad_y = grad_x.T
rgb_grad_x = np.stack([grad_x, grad_x, grad_x])
rgb_grad_y = np.stack([grad_y, grad_y, grad_y])
rgb_filters = np.stack([rgb_grad_x, rgb_grad_y])

conv.weight.data = torch.from_numpy(rgb_filters.astype(np.float32) )
print(conv.weight.data.shape)

In [ ]:
y = conv(x)
y.shape

In [ ]:
y_pil_0 = transforms.ToPILImage()(y.squeeze()[0,:,:])
y_pil_1 = transforms.ToPILImage()(y.squeeze()[1,:,:])

In [ ]:
display(transforms.ToPILImage()(train_x))
display(y_pil_0)
display(y_pil_1)

In [ ]:
conv_gaussian = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3) 
np_zeros = np.zeros((3,3), dtype=np.float32)
np_gauss = (1/16.)*np.asarray([[1,2,1],
                       [2,4,2],
                       [1,2,1]])
np_gauss = np_gauss.astype(np.float32)
r_gauss = np.stack([np_gauss, np_zeros, np_zeros])
g_gauss = np.stack([np_zeros, np_gauss, np_zeros])
b_gauss = np.stack([np_zeros, np_zeros, np_gauss])

rgb_gausses = np.stack([r_gauss, g_gauss, b_gauss])

conv_gaussian.weight.data = torch.from_numpy(rgb_gausses)





In [ ]:
y_smoothed = conv_gaussian(x)
y_smoothed.shape

In [ ]:
y_smoothed_r = transforms.ToPILImage()(y_smoothed.squeeze()[0,:,:])
# y_smoothed_r

In [ ]:
y_smoothed_g = transforms.ToPILImage()(y_smoothed.squeeze()[1,:,:])
# y_smoothed_g

In [ ]:
y_smoothed_b= transforms.ToPILImage()(y_smoothed.squeeze()[2,:,:])
# y_smoothed_b

In [ ]:
y = conv(y_smoothed)

In [ ]:
y_pil_0 = transforms.ToPILImage()(y.squeeze()[0,:,:])
y_pil_1 = transforms.ToPILImage()(y.squeeze()[1,:,:])

In [ ]:
display(y_pil_0)
display(y_pil_1)